In [3]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import random
import time
import undetected_chromedriver as uc
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd


In [ ]:
#Using ScraperAPI
payload = { 'api_key': '714ebc3e377d0996d657d9c45de9a426', 'url': 'https://www.reuters.com/world/us/' }
r = requests.get('https://api.scraperapi.com/', params=payload)

soup = BeautifulSoup(r.text, "html.parser")

article_links = set()

# Find all political article links 
for a in soup.find_all("a", href=True):
    href = a["href"]
    if any(word in href for word in ["/world/us/", "/business/", "/markets/", "/technology/", "/finance/", "/sustainability/"]) and href.count("/") > 3:
        article_links.add("https://www.reuters.com" + href)

print(article_links)
print(f"Total number of articles: {len(article_links)}")


{'https://www.reuters.com/world/us/trump-plans-executive-order-targeting-ticket-scalping-according-document-2025-03-31/', 'https://www.reuters.com/world/us/ntsb-investigate-close-call-between-delta-plane-air-force-jets-2025-03-31/', 'https://www.reuters.com/world/us/us-agency-leading-musks-firing-spree-hire-new-driver-emails-show-2025-03-31/', 'https://www.reuters.com/world/us/change-course-us-justice-dept-drops-challenge-georgia-voting-law-2025-03-31/', 'https://www.reuters.com/world/us/us-reviewing-federal-contracts-grants-harvard-over-antisemitism-allegations-2025-03-31/', 'https://www.reuters.com/world/us/us-supreme-court-hear-catholic-groups-bid-wisconsin-unemployment-tax-exemption-2025-03-31/', 'https://www.reuters.com/world/us/democratic-senators-urge-trump-revamp-china-trade-ties-revise-free-trade-deals-2025-03-31/', 'https://www.reuters.com/world/us/musk-gives-away-two-1-million-checks-wisconsin-voters-high-profile-judicial-race-2025-03-31/', 'https://www.reuters.com/world/us/

In [ ]:

"""
# Use Selenium to load the page and get the HTML content
driver = webdriver.Chrome() 
driver.get("https://www.reuters.com/world/us/")
time.sleep(3)  


soup = BeautifulSoup(driver.page_source, "html.parser")

article_links = set()

# Find all political article links 
for a in soup.find_all("a", href=True):
    href = a["href"]
     ("/legal/" in href and not any(x in href for x in ['/government/', '/transactional/', '/litigation/', '/us-supreme-court/', '/legalindustry/'])) or ("/world/us/" in href and href.count("/") > 3)
        article_links.add("https://www.reuters.com" + href)
        print(href)

print(article_links)
"""

'\n# Use Selenium to load the page and get the HTML content\ndriver = webdriver.Chrome() \ndriver.get("https://www.reuters.com/world/us/")\ntime.sleep(3)  \n\n\nsoup = BeautifulSoup(driver.page_source, "html.parser")\n\narticle_links = set()\n\n# Find all political article links \nfor a in soup.find_all("a", href=True):\n    href = a["href"]\n    if (("/legal/" in href and href.count("/") > 2) or ("/world/us/" in href and href.count("/"))):\n        article_links.add("https://www.reuters.com" + href)\n        print(href)\n\nprint(article_links)\n'

In [ ]:
""""
# Grab articles using UserAgent
ua = UserAgent()
options = uc.ChromeOptions()
options.add_argument(f"user-agent={ua.random}")
driver = uc.Chrome(options=options)


for a in article_links:
    try:
        driver.get(a)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testid="Heading"]'))
        )
        soup = BeautifulSoup(driver.page_source, "html.parser")
        title_tag = soup.find("span", attrs={"data-testid": "Heading"})
        title = title_tag.text.strip() if title_tag else "No title"
    except Exception as e:
        print(f"⚠️ Failed to scrape {a}: {e}")
        title = "Blocked or failed"
    
    # ✅ Print the title
    print(f"Title: {title}")
    time.sleep(random.uniform(5, 15))""
    """


SyntaxError: unterminated string literal (detected at line 25) (2372789278.py, line 25)

Titles:
Business: h1,  "Heading"

Text:
Business: div, paragraph-X

In [17]:
# Read in content from articles linked from the main page
data = [] 
for a in article_links:
    try:
        # Use ScraperAPI to scrape the article
        article_payload = {
            'api_key': '714ebc3e377d0996d657d9c45de9a426',
            'url': a
        }
        response = requests.get('https://api.scraperapi.com/', params=article_payload)
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract the title and text from the article page
        title_tag = soup.find("span", attrs={"data-testid": "Heading"}) if ("/legal/") in a else soup.find("h1", attrs={"data-testid": "Heading"})
        title = title_tag.text.strip() if title_tag else None
        paragraphs = soup.find_all("div", attrs={"data-testid": lambda x: x and x.startswith("paragraph-")})
        text = "\n".join(p.get_text(strip=True) for p in paragraphs)
        
        # Add the data to the list if title and text are found
        if title and text:
            data.append({"Title": title, "Text": text, "url": a}) 
    
    #Show if scrape fails
    except Exception as e:
        print(f"⚠️ Failed to scrape {a}: {e}")

#Save to CSV    
df = pd.DataFrame(data)
#df.to_csv("reuters_articles_v2.csv", index=False)

print(f"Scraped {len(data)} articles.")

    
    

Scraped 11 articles.
